In [ ]:
import sqlite3
from sqlite3 import Error

def connect(db_file):
    """ create a database connection to a SQLite database """
    conn = sqlite3.connect(db_file)
    return conn


In [ ]:
DB_FILE = "/tmp/company_db.db"

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select 1, SQLITE_VERSION();")
    res = c.fetchone()
    print(res)

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS Employee")
    c.execute("DROP TABLE IF EXISTS Task")
    res = c.fetchone()
    print(res)

In [ ]:
create_table = '''
CREATE TABLE IF NOT EXISTS Employee (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name text NOT NULL,
  sallary money NOT NULL
)
'''

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(create_table)

In [ ]:
insert_sql = '''
INSERT INTO Employee (name, sallary) VALUES (?,?)
''';

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(insert_sql, ("Jakub", 2500.00))
    c.execute(insert_sql, ("Michał", 3500.00))
    print(c.lastrowid)

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Employee")
    rows = c.fetchall() 
    for row in rows:
        print(row)


In [ ]:
create_tasks_table = """
    CREATE TABLE IF NOT EXISTS Task (
        id integer PRIMARY KEY,
        employee_id integer,
        name text NOT NULL,
        priority integer,
        FOREIGN KEY (employee_id) REFERENCES Employee(id)
);
"""

conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute(create_tasks_table)

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (10, "Learn DELETE FROM TABLE sql stmt", 100))

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Task")
    rows = c.fetchall() 
    for row in rows:
        print(row)

Note that Foreign keys constraints are not enabled by default in SQLite, you have to enable them first by the running the following command:

In [ ]:
enable_fk_check = '''PRAGMA foreign_keys = ON;'''
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(enable_fk_check)

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("PRAGMA foreign_keys")
    rows = c.fetchone() 
    for row in rows:
        print(row)

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (1, "Learn DELETE FROM TABLE sql stmt", 100))

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Task")
    rows = c.fetchall() 
    for row in rows:
        print(row)

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (100, "Learn DELETE FROM TABLE sql stmt", 100))

In [ ]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("""
        Select * from Employee e 
        JOIN Task t on e.id = t.employee_id;
    """)
    rows = c.fetchall() 
    for row in rows:
        print(row)